<a href="https://colab.research.google.com/github/bettercodepaul/data-wrangling-praktikum/blob/master/Dash_Part_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dash Introduction
Dash is a low-code framework for creating data-centric web applications, developed by Plotly. Dash aims to allow the user to work as much as possible in Python and is well-suited for use with Polars.

In this notebook, the basics of Dash are introduced through examples. The official documentation can be found here:

https://dash.plotly.com/

*Note: To ensure Dash works correctly in the Colab environment, JavaScript and cookies should be enabled in the browser.*

## Installation + Set-Up

In [ ]:
# Download utility module und data
import urllib.request
import os.path
UTILS_URL = "https://github.com/bettercodepaul/data-wrangling-praktikum/raw/master/utils.py"
urllib.request.urlretrieve(UTILS_URL, os.path.basename(UTILS_URL))
from utils import download_data, download_region_data
download_data()

In [ ]:
# install libraries
!pip install -qr requirements.txt

In [ ]:
# import polars
import polars as pl
# output up to 60 characters per column and do not abbreviate floating point numbers
pl.Config(fmt_str_lengths=60, fmt_float="full")

In [ ]:
# import exercises and utility functions
from utils import *
from exercises_en import *

In [ ]:
track_genres = pl.read_parquet("track-genres.parquet")
df = df = (
    pl.read_csv("spotify-charts-2017-2021-global-top200.csv.gz", try_parse_dates=True)
    .join(track_genres, on="url", how="left")
)
df.head(2)

In [ ]:
from dash import dcc, html, Dash, _dash_renderer, Input, Output
import plotly.express as px
import dash_mantine_components as dmc
# Set React version to 18.2.0 (required by Dash Mantine Components)
_dash_renderer._set_react_version("18.2.0")

## Static Apps

With Dash, very sophisticated interfaces with complex charts, selection and filter elements (e.g., dropdown menus, checkboxes, sliders, etc.), and layouts can be created.

Normally, Dash is run as a regular Python script, and access to the generated visualization is done through the browser. Within this notebook/Colab environment, however, we use the inline display.

In practice, you could easily package a Dash app into a container and run it, for example, in a Kubernetes cluster or cloud service.

To start, here is a minimal Dash application:

In [ ]:
app = Dash(__name__)

# The layout is defined similarly to HTML
app.layout = html.Div(children=[
    html.H1(children="Hello"),
    html.Div(children="This is your first Dash app.")
])

# The Dash app is displayed within the notebook itself (jupyter_mode="inline")
app.run(jupyter_mode="inline", jupyter_height=150)

For the layout in this notebook, we use **Dash Mantine Components** (https://www.dash-mantine-components.com). This does not change the concepts of Dash but makes the layout code much clearer in slightly more complex examples and brings more ready-made and consistently styled UI components.

In [ ]:
app = Dash(external_stylesheets=dmc.styles.ALL)

# We need to enclose all components in a MantineProvider
app.layout = dmc.MantineProvider([
     dmc.Title("Hello", order=1),
     dmc.Text("This is your first Dash app with Mantine.")
])

# The Dash app is displayed within the notebook itself (jupyter_mode="inline")
app.run(jupyter_mode="inline", jupyter_height=150)

To create charts for our data, we use the `plotly.express` module, imported above as `px`. It allows the creation of interactive charts in the browser (commonly used Matplotlib, on the other hand, only generates static charts): https://plotly.com/python/plotly-express/

In [ ]:
px.line(df.group_by("date").agg(pl.col("streams").sum()).sort("date"), x="date", y="streams", height=300)

With a `dcc.Graph`, we can also display the chart in a Dash app.

In [ ]:
app = Dash(external_stylesheets=dmc.styles.ALL)

app.layout = dmc.MantineProvider([
     dmc.Title("Dash-App mit Diagramm", order=5),
     # Diagram is passed with the attribute figure
     dcc.Graph(id="streams-chart", figure=px.line(df.group_by("date").agg(pl.col("streams").sum()).sort("date"), x="date", y="streams", height=300))
])

app.run(jupyter_mode="inline", jupyter_height=350)

### Excercise

Now it's your turn:



In [ ]:
q26.question()

In [ ]:
q26_df = ...

In [ ]:
q26.check(q26_df)

In [ ]:
q27.question()

In [ ]:
q27_fig = ...
q27_fig

In [ ]:
q27.check(q27_fig)

## Interactive Apps with Callbacks
A higher degree of interaction in Dash is achieved through callbacks. With callbacks, user inputs can be responded to, for example, to adjust another UI component like a chart when a selection is made in a dropdown component. Here is a minimal example:

In [ ]:
app = Dash(external_stylesheets=dmc.styles.ALL)

# For callbacks, the ids of the components are important
app.layout = dmc.MantineProvider([
    dmc.TextInput(id="my-input", value=None, label="Your Input", placeholder="Enter any text here..."),
    dmc.TextInput(id="my-output", value=None, label="My Output", disabled=True)
])


# The callback is called by the client as soon as a property of the input changes
# The return value of the callback is written by the client into the property of the output
@app.callback(
    Output(component_id="my-output", component_property="value"),
    Input(component_id="my-input", component_property="value")
)
def update_output_div(input_value):
    output_value = input_value
    return output_value

app.run(jupyter_mode="inline", jupyter_height=150)

Callbacks are annotated in Dash with `@app.callback` and the Output and Input parameters. The annotation links the callback function to the corresponding elements specified in the layout via ID and property values. All callbacks are executed once when the application starts to initialize elements.

<pre>
# For callbacks, the ids of the components are important
app.layout = dmc.MantineProvider([
     dmc.TextInput(id=<span style="color:#4169E1">"my-input"</span>, <span style="color:#4169E1">value</span>=<span style="color:white; background-color:#4169E1">None</span>, label="Your Input", placeholder="Enter any text here..."),
     dmc.TextInput(id=<span style="color:#C04000">"my-output"</span>, <span style="color:#C04000">value</span>=<span style="color:white; background-color:#C04000">None</span>, label="My Output", disabled=True)
])


# The callback is called by the client as soon as a property of the input changes
# The return value of the callback is written by the client into the property of the output
@app.callback(
    Output(component_id=<span style="color:#C04000">"my-output"</span>, component_property=<span style="color:#C04000">"value"</span>),
    Input(component_id=<span style="color:#4169E1">"my-input"</span>, component_property=<span style="color:#4169E1">"value"</span>)
)
def update_output_div(<span style="color:white; background-color:#4169E1">input_value</span>):
    output_value = input_value
    return <span style="color:white; background-color:#C04000">output_value</span>
</pre>

Properties specify the attributes of the layout elements that are relevant for the callback function. Multiple attributes can also be changed simultaneously in the output; an example of this will be provided at the end of the notebook.

Using callbacks, we will now build a simple filtering option:

In [ ]:
all_artists = df.select(pl.col("artist").str.split(", ")).get_column("artist").explode().unique().sort().to_list()

app = Dash(external_stylesheets=dmc.styles.ALL)

def get_streams_chart(artist):
    filter_expr =  pl.col("artist").str.contains(artist) if artist else pl.lit(True)
    data = df.filter(filter_expr).group_by("date").agg(pl.col("streams").sum()).sort("date")
    return px.line(data, x="date", y="streams", height=300, title=f"Daily Streams for {artist or 'all artists'}")

app.layout = dmc.MantineProvider([
    dmc.Title("Spotify Explorer", order=3, mb=20),
    dmc.Select(id="artist-select", label="Artist", placeholder="Select one", data=all_artists, w="400", searchable=True, clearable=True, mb=10),
    dcc.Graph(id="streams-chart", figure=get_streams_chart(None))
])

@app.callback(
    Output(component_id="streams-chart", component_property="figure"),
    Input(component_id="artist-select", component_property="value")
)
def update_streams_per_month(selected_artist):
    return get_streams_chart(selected_artist)

app.run(jupyter_mode="inline", jupyter_height=500)

### Multiple Inputs
We also want to filter by title.

Implementation in the Dash app with additional input components.

In [ ]:
all_artists = df.select(pl.col("artist").str.split(", ")).get_column("artist").explode().unique().sort().to_list()
all_titles = df.select(pl.col("title").unique()).get_column("title").sort().to_list()

app = Dash(external_stylesheets=dmc.styles.ALL)

def get_streams_chart(artist, title):
    filter_expr = pl.col("artist").str.contains(artist) if artist else pl.lit(True)
    filter_expr = filter_expr & pl.col("title").eq(title) if title else filter_expr
    data = df.filter(filter_expr).group_by("date").agg(pl.col("streams").sum()).sort("date")
    return px.line(data, x="date", y="streams", height=300, title=f"Daily Streams for {artist or 'all artists'} - {title or 'all titles'}")

app.layout = dmc.MantineProvider([
    dmc.Title("Spotify Explorer", order=3, mb=20),
    dmc.Group([
        dmc.Select(id="artist-select", label="Artist", placeholder="Select one", data=all_artists, searchable=True, clearable=True),
        dmc.Select(id="title-select", label="Title", placeholder="Select one", data=all_titles, searchable=True, clearable=True)
    ], grow=True, mb=10),
    dcc.Graph(id="streams-chart", figure=get_streams_chart(None, None))
])

@app.callback(
    Output(component_id="streams-chart", component_property="figure"),
    Input(component_id="artist-select", component_property="value"),
    Input(component_id="title-select", component_property="value")
)
def update_streams_per_month(selected_artist, selected_title):
    return get_streams_chart(selected_artist, selected_title)

app.run(jupyter_mode="inline", jupyter_height=500)

We can also restrict the displayed options based on the selected options.

In [ ]:
all_artists = df.select(pl.col("artist").str.split(", ")).get_column("artist").explode().unique().sort().to_list()
all_titles = df.select(pl.col("title").unique()).get_column("title").sort().to_list()

app = Dash(external_stylesheets=dmc.styles.ALL)

def get_streams_chart(artist, title):
    filter_expr = pl.col("artist").str.contains(artist) if artist else pl.lit(True)
    filter_expr = filter_expr & pl.col("title").eq(title) if title else filter_expr
    data = df.filter(filter_expr).group_by("date").agg(pl.col("streams").sum()).sort("date")
    return px.line(data, x="date", y="streams", height=300, title=f"Daily Streams for {artist or 'all artists'} - {title or 'all titles'}")

app.layout = dmc.MantineProvider([
    dmc.Title("Spotify Explorer", order=3, mb=20),
    dmc.Group([
        dmc.Select(id="artist-select", label="Artist", placeholder="Select one", data=all_artists, searchable=True, clearable=True),
        dmc.Select(id="title-select", label="Title", placeholder="Select one", data=all_titles, searchable=True, clearable=True)
    ], grow=True, mb=10),
    dcc.Graph(id="streams-chart", figure=get_streams_chart(None, None))
])

@app.callback(
    Output(component_id="streams-chart", component_property="figure"),
    Output(component_id="artist-select", component_property="data"), # possible artists for the current filter
    Output(component_id="title-select", component_property="data"), # possible titles for the current filter
    Input(component_id="artist-select", component_property="value"),
    Input(component_id="title-select", component_property="value")
)
def update_streams_per_month(selected_artist, selected_title):
    # determine possible titles for the current filter
    possible_titles = df.filter(pl.col("artist").str.contains(selected_artist) if selected_artist else pl.lit(True)).select(pl.col("title").unique()).get_column("title").sort().to_list()
    # determine possible artists for the current filter
    possible_artists = df.filter(pl.col("title").eq(selected_title) if selected_title else pl.lit(True)).select(pl.col("artist").str.split(", ")).get_column("artist").explode().unique().sort().to_list()
    return get_streams_chart(selected_artist, selected_title), possible_artists, possible_titles

app.run(jupyter_mode="inline", jupyter_height=500)

### Exercise

In [ ]:
q28.question()

In [ ]:
q28.hint()

## IFrame

In order to be able to listen to the songs, we integrate an IFrame containing the Spotify Player, that will play the most-streamed song.

In [ ]:
all_artists = df.select(pl.col("artist").str.split(", ")).get_column("artist").explode().unique().sort().to_list()
all_titles = df.select(pl.col("title").unique()).get_column("title").sort().to_list()

app = Dash(external_stylesheets=dmc.styles.ALL)

def filter_data(artist, title, rank_1_only):
    filter_expr = pl.col("artist").str.contains(artist) if artist else pl.lit(True)
    filter_expr = filter_expr & pl.col("title").eq(title) if title else filter_expr
    # consider all songs that have been at least once at #1
    filter_expr = filter_expr & pl.col("rank").min().over("artist", "title").eq(1) if rank_1_only else filter_expr
    return df.filter(filter_expr)

def get_streams_chart(filtered_data, title):
    data = filtered_data.group_by("date").agg(pl.col("streams").sum()).sort("date")
    return px.line(data, x="date", y="streams", height=300, title=title)

def get_top_song_player(filtered_data):
    top_song = filtered_data.group_by("url").agg(pl.col("streams").sum()).top_k(1, by="streams")
    return play_song(top_song)

app.layout = dmc.MantineProvider([
    dmc.Title("Spotify Explorer", order=3, mb=20),
    dmc.Group([
        dmc.Select(id="artist-select", label="Artist", placeholder="Select one", data=all_artists, searchable=True, clearable=True),
        dmc.Select(id="title-select", label="Title", placeholder="Select one", data=all_titles, searchable=True, clearable=True),
        # Switch for the option to show only songs that have been at least once at #1
        dmc.Switch(id="rank-1-switch", label="#1 hits only", checked=False)
    ], grow=True, mb=10),
    dcc.Graph(id="streams-chart", figure=get_streams_chart(df, "Daily Streams for all artists - 'all titles'")),
    dmc.Title("Most streamed title", order=4, mb=10),
    html.Iframe(id="player-iframe", src=None, width="100%", height="80", style={"border": "none"})
])

@app.callback(
    Output(component_id="streams-chart", component_property="figure"),
    Output(component_id="artist-select", component_property="data"),
    Output(component_id="title-select", component_property="data"),
    Output(component_id="player-iframe", component_property="src"),
    Input(component_id="artist-select", component_property="value"),
    Input(component_id="title-select", component_property="value"),
    Input(component_id="rank-1-switch", component_property="checked")
)
def update(selected_artist, selected_title, rank_1_only):
    # consider only songs that have been at least once at #1
    tmp_df = df.filter(pl.col("rank").eq(1)) if rank_1_only else df
    # determine possible titles for the current filter
    possible_titles = tmp_df.filter(pl.col("artist").str.contains(selected_artist) if selected_artist else pl.lit(True)).select(pl.col("title").unique()).get_column("title").sort().to_list()
    # determine possible artists for the current filter
    possible_artists = tmp_df.filter(pl.col("title").eq(selected_title) if selected_title else pl.lit(True)).select(pl.col("artist").str.split(", ")).get_column("artist").explode().unique().sort().to_list()
    filtered_data = filter_data(selected_artist, selected_title, rank_1_only)
    streams_chart_figure = get_streams_chart(filtered_data, f"Daily Streams for {selected_artist or 'all artists'} - {selected_title or 'all titles'}")
    player = get_top_song_player(filtered_data)
    return streams_chart_figure, possible_artists, possible_titles, player.src

app.run(jupyter_mode="inline", jupyter_height=550)

## Tables

We can also integrate a table, that shows an overview of the songs. We use the feature rich library AgGrid, that is also available for Dash: https://dash.plotly.com/dash-ag-grid

In [ ]:
import dash_ag_grid as dag

In [ ]:
all_artists = df.select(pl.col("artist").str.split(", ")).get_column("artist").explode().unique().sort().to_list()
all_titles = df.select(pl.col("title").unique()).get_column("title").sort().to_list()

app = Dash(external_stylesheets=dmc.styles.ALL)

def filter_data(artist, title, rank_1_only):
    filter_expr = pl.col("artist").str.contains(artist) if artist else pl.lit(True)
    filter_expr = filter_expr & pl.col("title").eq(title) if title else filter_expr
    # consider all songs that have been at least once at #1
    filter_expr = filter_expr & pl.col("rank").min().over("artist", "title").eq(1) if rank_1_only else filter_expr
    return df.filter(filter_expr)

def get_streams_chart(filtered_data, title):
    data = filtered_data.group_by("date").agg(pl.col("streams").sum()).sort("date")
    return px.line(data, x="date", y="streams", height=300, title=title)

def get_top_song_player(filtered_data):
    top_song = filtered_data.group_by("url").agg(pl.col("streams").sum()).top_k(1, by="streams")
    return play_song(top_song)

def get_grid_data(filtered_data):
    grid_data = (
        filtered_data
        .group_by("artist", "title")
        .agg(
            pl.col("streams").sum(),
            pl.col("rank").min().alias("highest rank")
        )
        .top_k(k=10, by="streams")
        .sort("streams", descending=True)
    )
    return grid_data 

def get_grid_component(grid_data):
    column_defs = [{"field": column_name} for column_name in grid_data.columns]
    return dag.AgGrid(
        id="top-songs-grid",
        rowData=grid_data.to_dicts(),
        columnDefs=column_defs,
        columnSize="autoSize",
    )

app.layout = dmc.MantineProvider([
    dmc.Title("Spotify Explorer", order=3, mb=20),
    dmc.Group([
        dmc.Select(id="artist-select", label="Artist", placeholder="Select one", data=all_artists, searchable=True, clearable=True),
        dmc.Select(id="title-select", label="Title", placeholder="Select one", data=all_titles, searchable=True, clearable=True),
        # Switch for the option to show only songs that have been at least once at #1
        dmc.Switch(id="rank-1-switch", label="#1 hits only", checked=False)
    ], grow=True, mb=10),
    dcc.Graph(id="streams-chart", figure=get_streams_chart(df, "Daily Streams for all artists - 'all titles'")),
    dmc.Title("Most streamed titles", order=4, mb=10),
    html.Iframe(id="player-iframe", src=None, width="100%", height="80", style={"border": "none"}),
    dmc.Space(h=10),
    get_grid_component(get_grid_data(df))
])

@app.callback(
    Output(component_id="streams-chart", component_property="figure"),
    Output(component_id="artist-select", component_property="data"),
    Output(component_id="title-select", component_property="data"),
    Output(component_id="player-iframe", component_property="src"),
    Output(component_id="top-songs-grid", component_property="rowData"),
    Input(component_id="artist-select", component_property="value"),
    Input(component_id="title-select", component_property="value"),
    Input(component_id="rank-1-switch", component_property="checked")
)
def update(selected_artist, selected_title, rank_1_only):
    # consider only songs that have been at least once at #1
    tmp_df = df.filter(pl.col("rank").eq(1)) if rank_1_only else df
    # determine possible titles for the current filter
    possible_titles = tmp_df.filter(pl.col("artist").str.contains(selected_artist) if selected_artist else pl.lit(True)).select(pl.col("title").unique()).get_column("title").sort().to_list()
    # determine possible artists for the current filter
    possible_artists = tmp_df.filter(pl.col("title").eq(selected_title) if selected_title else pl.lit(True)).select(pl.col("artist").str.split(", ")).get_column("artist").explode().unique().sort().to_list()
    filtered_data = filter_data(selected_artist, selected_title, rank_1_only)
    streams_chart_figure = get_streams_chart(filtered_data, f"Daily Streams for {selected_artist or 'all artists'} - {selected_title or 'all titles'}")
    player = get_top_song_player(filtered_data)
    grid_data = get_grid_data(filtered_data)
    return streams_chart_figure, possible_artists, possible_titles, player.src, grid_data.to_dicts()

app.run(jupyter_mode="inline", jupyter_height=1000)

# Final Open-topic Exercise

You now have various options:

- Extend the Spotify Explorer according to your ideas, for example...
    - Options for displaying and filtering by genres
    - Additional visualizations (trend of ranking, ...)
    - Additional filters by time (year, day, weekday, month, ...)
    - Additional information in the table (number of days at No. 1, in the Top 10, ...)
    - Additional column in the table where the song from the row can be played with a click
    - Get inspired by the [Mantine Components](https://www.dash-mantine-components.com)
    - Integrate a larger dataset with Top 200 per country
        - `download_region_data()`
        - `pl.read_parquet("spotify-charts-2017-2021.parquet").with_columns(pl.col("streams").cast(pl.Int64))`
- Design a Dash application that would help you in the project
- Install a Python environment on your local computer (see below)
- Continue working on the exercises for Polars, possibly with the third notebook
    - [Part 1](https://colab.research.google.com/github/bettercodepaul/data-wrangling-praktikum/blob/master/Polars_Part_1.ipynb) – Polars: Loading, Selecting, Filtering & Sorting
    - [Part 2](https://colab.research.google.com/github/bettercodepaul/data-wrangling-praktikum/blob/master/Polars_Part_2.ipynb) – Polars: Aggregations, Joins & Reshaping
    - [Part 3](https://colab.research.google.com/github/bettercodepaul/data-wrangling-praktikum/blob/master/Polars_Part_3.ipynb) – Polars: Custom Expressions, Lazy Mode and Streaming


## Further Resources

* Dash Gallery (Extensive collection of examples for Dash apps, including source code): https://dash-gallery.plotly.host/Portal/
* Local Python environment: uv (https://docs.astral.sh/uv/)
* Local Python IDE: VSCode (https://code.visualstudio.com/ with [Python Extension](https://marketplace.visualstudio.com/items?itemName=ms-python.python))
* Database access: SQLAlchemy (https://www.sqlalchemy.org/)
